In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# 1. 다음카페 로그인 페이지 열기

In [99]:
driver = webdriver.Chrome()
# 설치 폴더에 주의합니다. 

driver.get('https://logins.daum.net/accounts/loginform.do?mobilefull=1&category=cafe&url=http%3A%2F%2Fm.cafe.daum.net%2F_myCafe%3Fnull')
# 19년 5월부터 로그인 페이지 주소가 살짝 바뀌었네요. 

time.sleep(3)
# 페이지 전환시에는 적당한 시간을 줍니다. 
# 1. 과도한 크롤링 방지.
# 2. 페이지 전환이 완료되기 전에 다음 명령 실행되는 것 방지.
#    AJAX를 사용한 페이지는 페이지 전환시 딜레이가 꼭 필요한 경우도 있습니다.

# 2. 아이디, 패스워드 수동 입력

# 3. 카페 이름과 주소 지정

In [100]:
CAFE_NAME = 'rocksoccer' 
# 까페 이름을 넣어준다. 예제는 이종격투기... 

BOARD_NAME = 'ADs2' 
# 게시판 주소의 마지막 4자리(?)를 넣어준다.
# http://m.cafe.daum.net/ssaumjil/Jntr
QUERY = '쿠팡'

# 4. 락싸 국내토크방 게시판 이동 후 QUERY 검색

In [101]:
driver.get(f'http://m.cafe.daum.net/{CAFE_NAME}/{BOARD_NAME}/search?r=https%3A%2F%2Fm.cafe.daum.net%2Frocksoccer%2FADs2&query={QUERY}')
time.sleep(3)

# 이제 위에서 지정한 까페의 게시판으로 이동합니다. 
# 바로 게시물로 이동해도 됩니만.. 
# 계정정지의 압봵이 있으니... 게시판 리스트를 보고 게시물 하나만 소박하게 스크래핑하는 식으로... 

# 캡처

In [102]:
inp_num = input('저장할 게시물 번호 + 엔터: ')
# inp_num = '488816'
num = int(inp_num) # 뒤에 DB에 정수로 저장할 거니까 미리 형변환 해줍시다. 

url = 'http://m.cafe.daum.net/%s/%s/%s' % (CAFE_NAME, BOARD_NAME, inp_num)
driver.get(url) # 게시물의 주소로 이동합니다. 

time.sleep(3)

### 페이지 소스를 BeautifulSoup(이하 BS)에게 넘기는 것을 마지막으로 셀레니움의 역할은 끝이 납니다.

In [111]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser') 

# repr(soup)

### 제목 찾기

In [112]:
subject = soup.body.find('h3', class_='tit_subject')
# BS는 class명을 이용해서 h3을 바로 찾을 수 있습니다. 
# 클래스 명은 ID와 달리 중복이 가능합니다. 
# 태그와 클래스명까지 중복되어 겹치는 경우 첫번째 노드가 검색됩니다.  
# 다행히 이 HTML 문서에서 'tit_subject' 클래스는 한번만 사용됩니다. ^^ 
# 중복을 걱정할 필요가 없네요. 
# 중복 여부는 개발자 도구의 검색 기능을 이용해 보십시오

In [113]:
import pandas as pd
from bs4 import BeautifulSoup

# 본문과 댓글을 위한 별도의 DataFrame 초기화
df_columns_posts = ['num', 'subject', 'nick', 'write_time', 'views', 'url', 'contents']
df_posts = pd.DataFrame(columns=df_columns_posts)

df_columns_comments = ['cmt_num', 'cmt_writer', 'cmt_time', 'cmt_txt']
df_comments = pd.DataFrame(columns=df_columns_comments)

# 게시물 정보 추가 (게시물 본문 크롤링 부분에 추가)
if subject is None:
    print(url, '지워진 게시물입니다.')
else:
    subject = subject.get_text(strip=True)
    if soup.body.find('span', class_='txt_subject').find('span', class_='sr_only').get_text() == '작성자':
        nick = soup.body.find('span', class_='sr_only').next_sibling
    else:
        nick = ''
    num_subject = soup.body.find_all('span', class_='num_subject')
    write_time = num_subject[0].get_text()
    views = num_subject[1].get_text()
    contents = soup.body.find('div', id='article').get_text('\n', strip=True)
    
    # 게시물 정보를 df_posts에 추가
    new_row_post = {'num': inp_num, 'subject': subject, 'nick': nick, 'write_time': write_time, 'views': views, 'url': url, 'contents': contents}
    df_posts = pd.concat([df_posts, pd.DataFrame([new_row_post])], ignore_index=True)
    
    
    # 댓글
    cmt_total = soup.body.find('article', id='mArticle').find('span', class_="num_total").get_text()
    driver.get('http://m.cafe.daum.net/{0}/{1}/{2}/comments'.format(CAFE_NAME, BOARD_NAME, inp_num))
    time.sleep(1)
    # 페이지 숫자를 확인하기 위해 캡쳐
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # 페이지 숫자 확인
    cmt_page_max = int(soup.body.find('span', id="pagingNav").find_all('span', class_="num_page").pop().get_text())
    # 댓글 정보 추가 (댓글 크롤링 부분에 추가)
    for i in range(cmt_page_max):
        j = cmt_page_max-i
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        cmt_all = soup.body.find('ul', id="commentList").find_all('li')
        for k in cmt_all:
            # 기본값을 설정합니다
            cmt_reply = False
            reply_prefix = ''
            
            if k.div.span.find_all('span', class_="txt_bar"):
                if k['class'] == ['reply_on']:
                    cmt_reply = True
                    reply_prefix = 'ㄴ'
            
            # 삭제된 댓글 처리
            if k.find('strong', class_='tit_info'):
                if k.find('strong', class_='tit_info').find('span', class_='txt_detail').get_text(strip=True) == '삭제된 댓글입니다.':
                    continue
            
            cmt_num = inp_num + '_' + k['id'].split('comment_')[1]
            cmt_writer_elem = k.div.find('span', class_='username')
            if cmt_writer_elem:
                cmt_writer = cmt_writer_elem.get_text(strip=True)
            else:
                continue  # 삭제된 댓글이므로 건너뜁니다.
                
            cmt_time = k.div.span.find('span', class_="created_at").get_text()
            cmt_txt = k.div.find('span', class_='txt_detail').get_text(strip=True)
            
            # 댓글 정보를 df_comments에 추가
            new_row_comment = {'cmt_num': reply_prefix + cmt_num, 'cmt_writer': cmt_writer, 'cmt_time': cmt_time, 'cmt_txt': cmt_txt}
            df_comments = pd.concat([df_comments, pd.DataFrame([new_row_comment])], ignore_index=True)
        
        if j > 1:
            driver.get('http://m.cafe.daum.net/{0}/{1}/{2}/comments?prev_page={3}&mode=regular&cdepth={4}&page={5}'.format(CAFE_NAME, BOARD_NAME, inp_num, j, '0002100000', (j-1)))
            time.sleep(2)

In [114]:
df_posts

,num,subject,nick,write_time,views,url,contents
0,484256,그래도 쿠팡이 축구에 몰빵하는거 같아 기분좋네요,축구부장,24.04.28,"2,026",http://m.cafe.daum.net/rocksoccer/ADs2/484256,햄볶합니다\n이강인 김민재도 EPL로 왔으면..


In [115]:
df_comments

,cmt_num,cmt_writer,cmt_time,cmt_txt
0,484256_18,락싸야아프지마ㅠㅠ,24.04.28,별론가요?ㅋㅋ
1,484256_14,주세종,24.04.28,진짜 감사합니당
2,484256_16,NO.19 LEE KANG IN,24.04.28,오예
3,484256_19,주바페,24.04.28,오예
4,484256_24,로또,24.04.29,황덕연 떡상각인데 ㄷㄷ
5,484256_1,newoll,24.04.28,대박
6,484256_2,옴걸죠,24.04.28,1년 중계료가 700억ㄷㄷ
7,484256_3,아이스핫초코,24.04.28,어차피 리그1 분데스도 쿠팡중계 ㄷㄷ
8,484256_4,축구부장작성자 본인 여부작성자,24.04.28,하나만 보는게 더 편함 ㄷ
9,484256_5,화이트블랙,24.04.28,헐 그럼 스포티비는..
